# Creating interactive MapD components with IPython

MapD connects to the scipy ecosystem using the Ibis productivity framework and Jupyter Map D renderer.  These tools produce rich display representations of databases, dataframes and visualizations.  This notebook demonstrates these benefits using IPython widgets and magics to make MapD an interactive experience in the notebook.  

* The jupyter MapD renderer provides Jupyter Lab user components to interact with MapD. This notebook uses a running I Python kernel and widgets to establish interactive components

In [51]:
    import ibis.mapd; from ipywidgets import *; from traitlets import dlink, observe
    from IPython import display; from pandas import DataFrame, Series
    yaml = lambda object: __import__('yaml').safe_load(__import__('io').StringIO(object))

## Database connecions

Store the database connections in a configparser.

In [52]:
    parser = __import__('configparser').ConfigParser(defaults=yaml("""
    {host: localhost, port: '9091', user: mapd, password: HyperInteractive, protocol: binary, database: mapd}"""))

Add other databases, like the local immerse database provided by Docker hub

Note: you can start the docker server with `docker run -p 9090-9092:9090-9092 mapd/mapd-ce-cpu:v4.1.0`

In [53]:
    parser.read_dict(yaml("""default: {}
    immerse: {port: '9092', host: qs-dev.mapd.com, protocol: http}
    metis: {host: metis.mapd.com, port: '443', protocol: https}"""))

## Creating the widgets

Using `ipywidgets` container objects hold state for our connections.  The `tables` object preview information about the table.

In [54]:
    connections = HBox(children=[Dropdown(options=parser.sections(), value='default', description='Connection'), HTML()])
    tables = VBox(children=[Dropdown(description='tables'), HBox(), HTML()])

The widget will hold the state for our application.  The `get_info` function returns the current active selection as told by the `connections` object.

In [55]:
    def get_info(): info = parser[connections.children[0].value]; return {**info, 'port': int(info['port'])}

When a new connection is selected there will be a table that shares the connection information, and we reset our connection preview each time with `reset_tables`.

In [56]:
    def show_config(_): return reset_tables() or Series(dict(parser[_])).to_frame('').T.to_html()
    def reset_tables(): tables.children[0].options, tables.children[1].children, tables.children[2].value = [], [], ""

Once a new connection is established we `list_tables` that are in the connection.

In [57]:
    def list_tables(_):
        with ibis.mapd.connect(**get_info()) as con: return con.list_tables()

After establishing the possible tables we can use the Ibis repr and pandas DataFrame to provide a preview of the data.

In [58]:
    def show_table(_):
        try:
            tablename = tables.children[0].value
            with ibis.mapd.connect(**get_info()) as con:
                selection = con.table(tablename)
                img, table = HBox(children=[Output()]), HBox(children=[Output()])
                with img.children[0]:  display.display(selection)
                with table.children[0]:  display.display(selection.head().execute())
                tables.children[2].value = f"""<code>select * from {tablename}</code>"""
                return img, table
        except: return []
        

## Dynamic Linking

Dynamically linking the widget objects.

In [59]:
    dlink((connections.children[0], 'value'), (connections.children[1], 'value'), show_config)
    dlink((connections.children[0], 'value'), (tables.children[0], 'options'), list_tables)
    dlink((tables.children[0], 'value'), (tables.children[1], 'children'), show_table);

In [60]:
    manager = VBox(children=[connections, tables])
    __name__ == '__main__' and manager

In [70]:
    from IPython.core.magic import Magics, magics_class, register_line_cell_magic, cell_magic

    @magics_class
    class MapD(Magics):
        def __init__(self, shell):
            super().__init__(shell)
            self.connections, self.tables = connections, tables

        current_table = property(lambda x: x.tables.children[0].value)

        @cell_magic
        def mapd(self, line, cell=None):
            cell = yaml(cell)
            return {str: self.query, dict: self.view}[type(cell)](cell)

        def query(self, object):
            with ibis.mapd.connect(**get_info()) as con: return con.raw_sql(object, results=True).to_df()

        def view(self, object):
            m = __import__('mapd_renderer').MapDBackendRenderer({
                'dbName' if 'database' == _0 else _0 : _1 
                for _0, _1 in  get_info().items()}, object)
            print(m._repr_mimebundle(None, None))
            return m
        
   

* Register the extension.

In [62]:
    def load_ipython_extension(ip): ip.register_magics(MapD(get_ipython()))
    __name__ == '__main__' and load_ipython_extension(get_ipython())

In [64]:
    %%mapd
    select * from flights_donotmodify limit 4

,flight_year,flight_month,flight_dayofmonth,flight_dayofweek,deptime,crsdeptime,arrtime,crsarrtime,uniquecarrier,flightnum,...,dest_name,dest_city,dest_state,dest_country,dest_lat,dest_lon,origin_merc_x,origin_merc_y,dest_merc_x,dest_merc_y
0,2008,1,31,4,2029,2015,2121,2120,WN,2809,...,Dallas Love,Dallas,TX,USA,32.847115,-96.851768,-11321852.0,4193733.0,-10781490.0,3875028.50
1,2008,1,31,4,1304,1225,1404,1330,WN,1064,...,Theodore F Green State,Providence,RI,USA,41.723999,-71.428223,-8534665.0,4746827.5,-7951353.0,5119725.00
2,2008,1,31,4,613,615,728,710,WN,1628,...,William P Hobby,Houston,TX,USA,29.645418,-95.278893,-10781490.0,3875028.5,-10606397.0,3458052.75
3,2008,1,31,4,1118,1120,1219,1225,WN,1759,...,Albany Cty,Albany,NY,USA,42.748119,-73.802979,-8534665.0,4746827.5,-8215710.0,5273711.50


In [65]:
    %%mapd 
    SELECT goog_x as x, goog_y as y, tweets_nov_feb.rowid FROM tweets_nov_feb limit 10

,x,y,rowid
0,-8.244688e+06,4972038.00,12640
1,-7.885474e+06,5135954.50,13248
2,-8.585785e+06,4708023.00,12096
3,-1.351459e+07,4635878.50,6880
4,-4.885402e+06,-1892679.75,8096
5,-7.654110e+06,-3882334.50,12097
6,-5.693870e+06,-2669869.00,8097
7,2.055925e+06,-4021811.00,12098
8,-1.187361e+07,4155311.75,13024
9,-3.500757e+05,7194443.00,6881


## Visualization

The `mapd` magic may accept yaml in the body.  When `yaml` is used we return a visualization.

In [67]:
import mapd_renderer

In [71]:
%%mapd

width: 384
height: 564
config: {ticks: false}
data:
  - name: 'tweets'
    sql: 'SELECT goog_x as x, goog_y as y, tweets_nov_feb.rowid FROM tweets_nov_feb' 
scales:
  - name: 'x'
    type: 'linear'
    domain: [3650484.1235206556, 7413325.514451755]
    range: 'width'
  - name: 'y'
    type: 'linear'
    domain: [5778161.9183506705, 10471808.487466192]
    range: 'height'
marks:
  - type: 'points'
    from: {data: 'tweets'}
    properties:
      x: {scale: 'x', field: 'x'}
      y: {scale: 'y', field: 'y'}
      fillColor: 'green'
      size: {value: 1}